# GHZ Game Workbook

**What is this workbook?**
A workbook is a collection of problems, accompanied by solutions to them. 
The explanations focus on the logical steps required to solve a problem; they illustrate the concepts that need to be applied to come up with a solution to the problem, explaining the mathematical steps required. 

Note that a workbook should not be the primary source of knowledge on the subject matter; it assumes that you've already read a tutorial or a textbook and that you are now seeking to improve your problem-solving skills. You should attempt solving the tasks of the respective kata first, and turn to the workbook only if stuck. While a textbook emphasizes knowledge acquisition, a workbook emphasizes skill acquisition.

This workbook describes the solutions to the problems offered in the [GHZ Game kata](./GHZGame.ipynb). 
Since the tasks are offered as programming problems, the explanations also cover some elements of Q# that might be non-obvious for a first-time user.

To begin, first prepare this notebook for execution (if you skip this step, you'll get "Syntax does not match any known patterns" error when you try to execute Q# code in the next cells):

In [ ]:
%package Microsoft.Quantum.Katas::0.12.20100202-beta

> The package versions in the output of the cell above should always match. If you are running the Notebooks locally and the versions do not match, please install the IQ# version that matches the version of the `Microsoft.Quantum.Katas` package.
> <details>
> <summary><u>How to install the right IQ# version</u></summary>
> For example, if the version of `Microsoft.Quantum.Katas` package above is 0.1.2.3, the installation steps are as follows:
>
> 1. Stop the kernel.
> 2. Uninstall the existing version of IQ#:
>        dotnet tool uninstall microsoft.quantum.iqsharp -g
> 3. Install the matching version:
>        dotnet tool install microsoft.quantum.iqsharp -g --version 0.1.2.3
> 4. Reinstall the kernel:
>        dotnet iqsharp install
> 5. Restart the Notebook.
> </details>


## Part I. Classical GHZ


### Task 1.1. Win Condition
**Inputs:** 

  1. Alice, Bob and Charlie's input bits (r, s and t), stored as an array of length 3,

  2. Alice, Bob and Charlie's output bits (a, b and c), stored as an array of length 3.

The input bits will have zero or two bits set to true.

**Output:** 
True if Alice, Bob and Charlie won the GHZ game, that is, if r ∨ s ∨ t = a ⊕ b ⊕ c, and false otherwise.

### Solution

There are four inputs possible, (0,0,0), (0,1,1), (1,0,1), and (1,1,0), each with 25% probability.
Therefore, in order to win, the sum of the output bits has to be even if the input is (0,0,0) and odd otherwise.

To check whether the win condition holds, you need to compute the expressions $r \vee s \vee t$ and $a \oplus b \oplus c$ and to compare them: if they are equal, the game is won. To compute the expressions, you can use either logical functions `Or` and `Xor` from the [`Microsoft.Quantum.Logical`](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.logical) library.

In [ ]:
%kata T11_WinCondition_Test 

open Microsoft.Quantum.Logical;

function WinCondition (rst : Bool[], abc : Bool[]) : Bool {
    let p = Or(rst[0], Or(rst[1], rst[2])); 
    let u = Xor(abc[0], Xor(abc[1], abc[2]));
    return (p == u);
}

[Return to task 1.1 of the GHZ Game kata.](./GHZGame.ipynb#Task-1.1.-Win-Condition)

### Task 1.2. Random classical strategy

**Input:** The input bit for one of the players (r, s or t).

**Output:** A random bit that this player will output (a, b or c).

If all players use this strategy, they will win about 50% of the time.

### Solution

Q# namespace [Microsoft.Quantum.Random](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.random) offers a variety of operations implementing random number generation. [DrawRandomBool](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.random.drawrandombool) is particularly convenient for this task.

In [ ]:
%kata T12_RandomClassical_Test 

open Microsoft.Quantum.Random;

operation RandomClassicalStrategy (input : Bool) : Bool {
    return DrawRandomBool(0.5);
}

[Return to task 1.2 of the GHZ Game kata.](./GHZGame.ipynb#Task-1.2.-Random-classical-strategy)

### Task 1.3. Best classical strategy

**Input:** The input bit for one of the players (r, s or t).

**Output:** A bit that this player will output (a, b or c) to maximize their chance of winning.

All players will use the same strategy.
The best classical strategy should win about 75% of the time.

### Solution

If all three players return TRUE, then a ⊕ b ⊕ c = TRUE by necessity (since the sum of their bits is odd).
This will win against inputs 011, 101, and 110 and lose against 000.
Since the four above inputs have equal probability and represent all possible inputs,
this strategy wins with 75% probability.

In [ ]:
%kata T13_BestClassical_Test 

operation BestClassicalStrategy (input : Bool) : Bool {
    return true;
}

[Return to task 1.3 of the GHZ Game kata.](./GHZGame.ipynb#Task-1.3.-Best-classical-strategy)

### Task 1.4. Referee classical GHZ game

**Inputs:** 

  1. An operation which implements a classical strategy (i.e., takes an input bit and produces an output bit),

  2. An array of 3 input bits that should be passed to the players.

**Output:** 
An array of 3 bits that will be produced if each player uses this strategy.

### Solution

You are given both the input bits and the strategy each of the players are using, so you have simply to convert them to the output bits and return those.

In [ ]:
%kata T14_PlayClassicalGHZ_Test 

operation PlayClassicalGHZ (strategy : (Bool => Bool), inputs : Bool[]) : Bool[] {
    let r = inputs[0];
    let s = inputs[1];
    let t = inputs[2];
    let a = strategy(r);
    let b = strategy(s);
    let c = strategy(t);
    return [a, b, c];
}

You can make the code a lot more concise using library operation [ForEach](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.arrays.foreach) that applies an operation to each element of an array and returns an array of results:

In [ ]:
%kata T14_PlayClassicalGHZ_Test 

open Microsoft.Quantum.Arrays;

operation PlayClassicalGHZ (strategy : (Bool => Bool), inputs : Bool[]) : Bool[] {
    return ForEach(strategy, inputs);
}

[Return to task 1.4 of the GHZ Game kata.](./GHZGame.ipynb#Task-1.4.-Referee-classical-GHZ-game)

## Part II. Quantum GHZ

In the quantum version of the game, the players still can not
communicate during the game, but they are allowed to share 
qubits from an entangled triple before the start of the game.


### Task 2.1. Entangled triple

**Input:** An array of three qubits in the $|000\rangle$ state.

**Goal:** Create the entangled state $|\Phi\rangle = \frac{1}{2} \big(|000\rangle - |011\rangle - |101\rangle - |110\rangle \big)$ on these qubits.

### Solution

1. Apply an X gate to the first and the second qubits to get the $|110\rangle$ state.
2. Appy an H gate to the first and the second qubits to get the following state:
$$\frac12 \big( |000\rangle - |010\rangle - |100\rangle + |110\rangle \big)$$
3. Flip the sign of the last term using a controlled Z gate with the first qubit as control and the second qubit as target (or vice versa):
$$\frac12 \big( |000\rangle - |010\rangle - |100\rangle \color{blue}- |110\rangle \big)$$
4. Now we have the right signs for each term, and the first and the last terms match those of the state we're preparing, so we just need to adjust the two middle terms. 
To do this, we can use [ControlledOnBitString](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.canon.controlledonbitstring) operation to flip the state of the last qubit if the first two qubits are in $|01\rangle$ or in $|10\rangle$ states, which gives us:
$$\frac{1}{2} \big(|000\rangle - |01\color{blue}1\rangle - |10\color{blue}1\rangle - |110\rangle \big)$$

In [ ]:
%kata T21_CreateEntangledTriple_Test 

operation CreateEntangledTriple (qs : Qubit[]) : Unit {
    X(qs[0]);
    X(qs[1]);

    H(qs[0]);
    H(qs[1]);

    CZ(qs[0], qs[1]);

    (ControlledOnBitString([false, true], X))([qs[0], qs[1]], qs[2]);
    (ControlledOnBitString([true, false], X))([qs[0], qs[1]], qs[2]);
}

[Return to task 2.1 of the GHZ Game kata.](./GHZGame.ipynb#Task-2.1.-Entangled-triple)

### Task 2.2. Quantum strategy

**Inputs:**

  1. The input bit for one of the players (r, s or t),

  2. That player's qubit of the entangled triple shared between the players.

**Goal:** Measure the qubit in the Z basis if the bit is 0 (false), or the X basis if the bit is 1 (true), and return the result.

The state of the qubit after the operation does not matter.

### Solution

In Q#, you can perform measurements in a specific basis using either the 
[Measure operation](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.intrinsic.measure) 
or convenient shorthands for measure-and-reset-to-$|0\rangle$ sequence of operations 
[MResetZ](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.measurement.mresetz) and 
[MResetX](https://docs.microsoft.com/qsharp/api/qsharp/microsoft.quantum.measurement.mresetx).

In [ ]:
%kata T22_QuantumStrategy_Test 

open Microsoft.Quantum.Measurement;

operation QuantumStrategy (input : Bool, qubit : Qubit) : Bool {
    if (input) {
        let res = MResetX(qubit);
        return res == One;
    }
    else {
        let res = MResetZ(qubit);
        return res == One;
    }
}

Alternatively, you can recall that measuring the qubit in the X basis is equivalent to applying an H gate to it and measuring it in the Z basis.

In [ ]:
%kata T22_QuantumStrategy_Test 

operation QuantumStrategy (input : Bool, qubit : Qubit) : Bool {
    if (input) {
        H(qubit);
    }
    return M(qubit) == One;
}

[Return to task 2.2 of the GHZ Game kata.](./GHZGame.ipynb#Task-2.2.-Quantum-strategy)

### Discussion: Why the above strategy has a 100% win rate

Recall the formula for the win condition:
* The sum of the answer bits must be even if the question bits are (0,0,0)
* The sum of the answer bits must be odd if the question bits are (1,1,0), (1,0,1) or (0,1,1).

> As a reminder, the probability "wavefunction" for three qubits is given by the following vector of length 8:
>
> $$
\begin{bmatrix}
\psi_{000}\\
\psi_{001}\\
\psi_{010}\\
\psi_{011}\\
\psi_{100}\\
\psi_{101}\\
\psi_{110}\\
\psi_{111}
\end{bmatrix}
$$
>
> $|\psi_{ijk}|^2$ gives the probability of observing the corresponding basis state $|ijk\rangle$ upon measuring the qubit trio.

Now, the entangled state $|\Phi\rangle$ that Alice, Bob and Charlie have agreed to use is represented as

$$
\begin{bmatrix}
+1/2\\
 0\\
 0\\
-1/2\\
 0\\
-1/2\\
-1/2\\
 0
\end{bmatrix}
$$

Let's first consider the case in which **all three players got the 0 bit**.

When the players make their measurements, they will collectively get one of the basis states of the original state - 000, 011, 101 or 110. This measn they'll report back zero "1" bits between them (with 25% probability) or two "1" bits between them (with 75% probability), either way satisfying the win condition for the team.

Now, suppose **Alice gets a 0 bit and the others get 1**.

Alice, looking at the 0, takes a Z basis measurement as before, while Bob and Charlie each take X basis measurements. 
(An X basis measurement is also equivalent to performing a Hadamard transform followed by a standard Z basis measurement, as the X basis is the $| + \rangle$ / $| - \rangle$, and a Hadamard transform rotates the $| 0 \rangle$ / $| 1 \rangle$ basis to $| + \rangle$ / $| - \rangle$.) 
So Bob and Charlie apply a Hadamard transform to their qubits, which corresponds to the following transformation applied to the whole system state:

$$
I \otimes H \otimes H = \begin{bmatrix}
1/2 & 1/2 & 1/2 & 1/2 & 0 & 0 & 0 & 0\\
1/2 & -1/2 & 1/2 & -1/2 & 0 & 0 & 0 & 0\\
1/2 & 1/2 & -1/2 & -1/2 & 0 & 0 & 0 & 0\\
1/2 & -1/2 & -1/2 & 1/2 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 1/2 & 1/2 & 1/2 & 1/2\\
0 & 0 & 0 & 0 & 1/2 & -1/2 & 1/2 & -1/2\\
0 & 0 & 0 & 0 & 1/2 & 1/2 & -1/2 & -1/2\\
0 & 0 & 0 & 0 & 1/2 & -1/2 & -1/2 & 1/2
\end{bmatrix}
$$

When applied to the original entangled state, all the amplitude shifts to the states corresponding to $|001\rangle$, $|010\rangle$, $|100\rangle$, and $|111\rangle$.  The precise configuration of the new entangled state is

$$
\begin{bmatrix}
 0\\
 1/2\\
 1/2\\
 0\\
-1/2\\
 0\\
 0\\
 1/2
\end{bmatrix}
$$

Now the players perform their measurements, and an odd number of them will see "1" (thanks to the new entangled state), again satisfying the win condition.  

Similarly, if **Alice and Charlie get "1" bits and Bob a "0"**, Alice and Charlie will apply Hadamard transforms to their qubits to give the tensor product

$$
H \otimes I \otimes H = \begin{bmatrix}
1/2 & 1/2  & 0   & 0    & 1/2  & 1/2  & 0    & 0\\
1/2 & -1/2 & 0   & 0    & 1/2  & -1/2 & 0    & 0\\
0   & 0    & 1/2 & 1/2  & 0    & 0    & 1/2  & 1/2\\
0   & 0    & 1/2 & -1/2 & 0    & 0    & 1/2  & -1/2\\
1/2 & 1/2  & 0   & 0    & -1/2 & -1/2 & 0    & 0\\
1/2 & -1/2 & 0   & 0    & -1/2 & 1/2  & 0    & 0\\
0   & 0    & 1/2 & 1/2  & 0    & 0    & -1/2 & -1/2\\
0   & 0    & 1/2 & -1/2 & 0    & 0    & -1/2 & 1/2
\end{bmatrix}
$$

The resulting state vector before the measurement will be the same as in the previous case, except that the $|010\rangle$ state ends up with the negative amplitude instead of $|100\rangle$.  Again the players report back an odd number of true bits between them and the team wins.

Finally if Charlie got the "0" bit and Alice and Bob both got "1", the latter two will apply Hadamard transform for the tensor product

$$
H \otimes H \otimes I = \begin{bmatrix}
1/2 & 0 & 1/2 & 0 & 1/2 & 0 & 1/2 & 0\\
0 & 1/2 & 0 & 1/2 & 0 & 1/2 & 0 & 1/2\\
1/2 & 0 & -1/2 & 0 & 1/2 & 0 & -1/2 & 0\\
0 & 1/2 & 0 & -1/2 & 0 & 1/2 & 0 & -1/2\\
1/2 & 0 & 1/2 & 0 & -1/2 & 0 & -1/2 & 0\\
0 & 1/2 & 0 & 1/2 & 0 & -1/2 & 0 & -1/2\\
1/2 & 0 & -1/2 & 0 & -1/2 & 0 & 1/2 & 0\\
0 & 1/2 & 0 & -1/2 & 0 & -1/2 & 0 & 1/2
\end{bmatrix}
$$

Operating with this on the original entangled state yields $(|100\rangle + |010\rangle - |001\rangle + |111\rangle)/2$ and once more the team will report back an odd number of true bits between them and win.

[Return to task 2.2 of the GHZ Game kata.](./GHZGame.ipynb#Task-2.2.-Quantum-strategy)

### Task 2.3. Play the GHZ game using the quantum strategy

**Input:** Operations that return Alice, Bob and Charlie's output bits (a, b and c) based on
their quantum strategies and given their respective qubits from the entangled triple.
The players have already been told what their starting bits (r, s and t) are.

**Goal:** Return an array of players' output bits (a, b and c).

### Solution

Putting together the building blocks we've implemented into a strategy is very simple:

1. Allocate three qubits and prepare our entangled state on them (using `CreateEntangledTriple` from task 2.1).
2. Send one of the qubits to each of the players (this step is "virtual", not directly reflected in Q# code, other than making sure that the strategies each act on their qubit only).
3. Have the players perform their measurements on their respective qubits using corresponding elements of the `strategies` array.
4. Return their measurement results.

In [ ]:
%kata T23_PlayQuantumGHZ_Test 

operation PlayQuantumGHZ (strategies : (Qubit => Bool)[]) : Bool[] {
    using (ghz = Qubit[3]) {
        CreateEntangledTriple(ghz);

        let a = strategies[0](ghz[0]);
        let b = strategies[1](ghz[1]);
        let c = strategies[2](ghz[2]);

        return [a, b, c];
    }
}

[Return to task 2.3 of the GHZ Game kata.](./GHZGame.ipynb#Task-2.3.-Play-the-GHZ-game-using-the-quantum-strategy)